In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [6]:
data = pd.read_csv('Online Retail-unclean.csv')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [7]:
missing_data = data.isnull().sum()

In [8]:
missing_columns = missing_data[missing_data > 0]
print(missing_columns)

Description      1454
UnitPrice           5
CustomerID     135086
Country             8
dtype: int64


In [9]:
missing_info = data[data.isnull().any(axis=1)]

In [10]:
missing_counts = missing_info.isnull().sum(axis=1)

In [11]:
missing_info['MissingCount'] = missing_counts
grouped_missing = missing_info.groupby('Country')['MissingCount'].sum()

for country, count in grouped_missing.items():
    print(f"Negara yang memiliki missing data adalah '{country}'")

Negara yang memiliki missing data adalah 'Australia'
Negara yang memiliki missing data adalah 'Bahrain'
Negara yang memiliki missing data adalah 'Canada'
Negara yang memiliki missing data adalah 'EIRE'
Negara yang memiliki missing data adalah 'France'
Negara yang memiliki missing data adalah 'Hong Kong'
Negara yang memiliki missing data adalah 'Israel'
Negara yang memiliki missing data adalah 'Portugal'
Negara yang memiliki missing data adalah 'Singapore'
Negara yang memiliki missing data adalah 'Switzerland'
Negara yang memiliki missing data adalah 'United Kingdom'
Negara yang memiliki missing data adalah 'Unspecified'


In [12]:
data = data.dropna(subset=['Description'])

In [13]:
data['UnitPrice'].fillna(data['UnitPrice'].mean(), inplace=True)

In [14]:
data['CustomerID'].fillna(0, inplace=True)

In [15]:
country_mode = data['Country'].mode()[0]

In [16]:
data['Country'].fillna(country_mode, inplace=True)

In [17]:
missing_data = data.isnull().sum()

In [18]:
print("Jumlah missing data per kolom:\n", missing_data)

Jumlah missing data per kolom:
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [19]:
if missing_data.sum() > 0:
    print("Masih ada missing data di file.")
else:
    print("Tidak ada missing data di file.")

Tidak ada missing data di file.


In [20]:
data.to_csv('cleaned_data.csv', index=False)

In [21]:
mean_unitprice = data['UnitPrice'].mean()

In [22]:
print("Mean dari kolom 'UnitPrice':", mean_unitprice)

Mean dari kolom 'UnitPrice': 4.623501663428626


In [23]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [24]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [25]:
data['Month'] = data['InvoiceDate'].dt.to_period('M') 

In [26]:
aggregated_data = data.groupby(['Country', 'Month'])['UnitPrice'].sum().reset_index()

In [27]:
print(aggregated_data)

         Country    Month   UnitPrice
0      Australia  2010-12  147.507003
1      Australia  2011-01  339.410505
2      Australia  2011-02  273.630000
3      Australia  2011-03  372.570000
4      Australia  2011-04  396.550000
..           ...      ...         ...
309  Unspecified  2011-06   42.100000
310  Unspecified  2011-07  627.050000
311  Unspecified  2011-08  146.510000
312  Unspecified  2011-09  108.430000
313  Unspecified  2011-11  116.100000

[314 rows x 3 columns]


In [31]:
aggregated_data.to_csv('Aggregasi.csv', index=False)

In [29]:
import math

country_counts = data['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'TotalData']

def calculate_sample_size(total, alpha):
    Z = {0.03: 1.88, 0.01: 2.58}[alpha]  
    p = 0.5  
    E = alpha  
    n = (Z*2 * p * (1 - p)) / (E*2)
    n = math.ceil(n)  
    return min(total, n)  

country_counts['Sample_alpha_0.03'] = country_counts['TotalData'].apply(lambda x: calculate_sample_size(x, 0.03))
country_counts['Sample_alpha_0.01'] = country_counts['TotalData'].apply(lambda x: calculate_sample_size(x, 0.01))

country_counts.to_csv('Sampling.csv', index=False)

print(country_counts)

                 Country  TotalData  Sample_alpha_0.03  Sample_alpha_0.01
0         United Kingdom     494032                 16                 65
1                Germany       9495                 16                 65
2                 France       8557                 16                 65
3                   EIRE       8196                 16                 65
4                  Spain       2533                 16                 65
5            Netherlands       2371                 16                 65
6                Belgium       2069                 16                 65
7            Switzerland       2002                 16                 65
8               Portugal       1519                 16                 65
9              Australia       1256                 16                 65
10                Norway       1086                 16                 65
11                 Italy        803                 16                 65
12       Channel Islands        758   